In [25]:
from typing import TypedDict, Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,BaseMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, END, add_messages

In [26]:
model = ChatOpenAI(
    base_url="https://ws-02.wade0426.me/v1",
    api_key="day3hw",
    model="Qwen/Qwen3-VL-8B-Instruct",
    temperature=0.1
)

In [27]:
class State(TypedDict):
    original_text: str
    translated_text: str
    critique: str
    attempts: int

In [28]:
def translator_node(state: State):
    # 這裡的 state['attempts'] 是從 LangGraph 狀態中讀取的
    current_attempt = state['attempts'] + 1
    print(f"\n--- 翻譯嘗試 (第{current_attempt}次) ---")

    prompt = f"你是一名翻譯員，請將以下中文翻譯成英文，不需任何解釋： '{state['original_text']}'"

    if state.get('critique'): # 使用 .get 比較安全，防止 key 不存在
        prompt += f"\n\n上一輪的審查意見是： '{state['critique']}'。請根據意見修正翻譯。"

    # model.invoke 回傳的是 AIMessage 物件
    response = model.invoke([HumanMessage(content=prompt)])

    # 1. 使用 response.content 取得翻譯文字
    # 2. 從 state['attempts'] 累加次數
    return {
        "translated_text": response.content,
        "attempts": current_attempt
    }

In [29]:
def reflector_node(state: State):
    print("---  審查中 ---")
    print(f"翻譯： {state['translated_text']}")
    prompt = f"""
            你是一個嚴格的翻譯審查員。
            原文： {state['original_text']}
            翻譯： {state['translated_text']}

            請檢查翻譯是否準確且通順。
            - 如果翻譯很完美，請只回覆 "Pass"
            - 如果需要修改，請給出簡短的具體建議。
            """

    response = model.invoke([HumanMessage(content=prompt)])

    return {"critique": response.content}

In [30]:
def should_continue(state: State) -> Literal["translator", "end"]:
    critique = state["critique"].strip().upper()

    if "PASS" in critique:
        print("--- 審查通過！ ---")
        return "end"
    elif state["attempts"] >= 3:
        print(" --- 達到最大重試次數，強制結束 ---")
        return "end"
    else:
        print(f"--- 審查未通過： {state['critique']} ---")
        print("--- 退回重寫 ---")
        return "translator"

In [31]:
workflow = StateGraph(State)

workflow.add_node("translator", translator_node)
workflow.add_node("reflector", reflector_node)

workflow.set_entry_point("translator")

workflow.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "translator": "translator",
        "end": END
    }
)

app = workflow.compile()
print(app.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
+------------+ 
| translator | 
+------------+ 
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


In [ ]:
if __name__ == "__main__":
    while True:
        user_input = input("\nUser: ")
        if user_input.lower() in ["exit", "quit"]: break
        inputs = {
            "original_text": user_input,
            "attempts": 0,
            "critique": ""
        }

        result = app.invoke(inputs)
        print("\n====== 最終結果 =======")
        print(f"原文： {result['original_text']}")
        print(f"最終翻譯： {result['translated_text']}")
        print(f"最終次數： {result['attempts']}")